Retention Redshift Insert (API Method)
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* APr 5, 2022

In [11]:
import boto3
import getpass
import pandas as pd
import awswrangler as wr
import pyodbc

from datetime import datetime
from pytz import timezone


### Let's connect to Redshift QA-APP to get our dataset:

In [12]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

RDMV-BMVB
Successully logged into Start URL: https://stellaralgo.awsapps.com/start#/


In [13]:
try:
    session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
    client = boto3.client('redshift')

    connqa = wr.data_api.redshift.connect(
        cluster_id="qa-app",
        database="datascience",
        db_user="admin"
    )
    
    print('QA CREDENTIALS RETRIEVED SUCCESSFULLY')
    
except Exception as err:
    print('FAILED TO RETRIEVE QA CREDENTIALS!', err)

QA CREDENTIALS RETRIEVED SUCCESSFULLY


In [14]:
qacode =  f"""
    SELECT *
    FROM 
       datascience.ds.datadictionary
    """
df = wr.data_api.redshift.read_sql_query(
            sql = qacode, 
            con = connqa
        )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   featureimportanceid  88 non-null     int64 
 1   feature              88 non-null     object
 2   featureraw           88 non-null     object
 3   datatype             88 non-null     object
 4   source               88 non-null     object
 5   engineered           88 non-null     object
 6   description          88 non-null     object
dtypes: int64(1), object(6)
memory usage: 4.9+ KB


### Now we can create another session to RedShift PROD-APP, to inser data:

In [15]:
try:
    session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
    client = boto3.client('redshift')

    connprod = wr.data_api.redshift.connect(
        cluster_id="prod-app",
        database="datascience",
        db_user="admin"
    )
    
    print('PROD CREDENTIALS RETRIEVED SUCCESSFULLY')
    
except Exception as err:
    print('FAILED TO RETRIEVE PROD CREDENTIALS!', err)

PROD CREDENTIALS RETRIEVED SUCCESSFULLY


### Insert the new scores into the datadictionary table:

In [10]:
try:
    fields = f"""
    INSERT INTO datascience.ds.datadictionary (
        featureimportanceid,
        feature,
        featureraw,
        datatype,
        source,
        engineered,
        description
    ) VALUES """

    # Insert Dataframe into SQL Server:
    values_list = []
    for i, (index, row) in enumerate(df.iterrows()):

        values = f"""
            (
                {str(row.featureimportanceid)},
                '{str(row.feature)}',
                '{str(row.featureraw)}',
                '{str(row.datatype)}',
                '{str(row.source)}',
                {str(row.engineered)},
                '{str(row.description)}'
            )"""

        values_list.append(values)
        
        insert_statement = fields + ",".join(values_list)+";"

        wr.data_api.redshift.read_sql_query(
            sql = insert_statement, 
            con = connprod
        )

except Exception as err:
    print("ERROR WHILE INSERTING DATA DICTIONARY TO DATABASE:", err)

print("DATA DICTIONARY INSERTS COMPLETED")

TEAMS CONFIG INSERTS COMPLETED
